<a href="https://colab.research.google.com/github/aarushkachhawa/Pinecone-Playground/blob/main/pinecone_api_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pinecone-client
from pinecone import Pinecone, ServerlessSpec
import pandas as pd

from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Pinecone-Playground

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 3.1 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/My Drive/Pinecone-Playground
